<a href="https://colab.research.google.com/github/anitakumar/ml/blob/main/ags_adalora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

AdaLoRA is a method for optimizing the number of trainable parameters to assign to weight matrices and layers, unlike LoRA, which distributes parameters evenly across all modules.AdaLoRA is a method for optimizing the number of trainable parameters to assign to weight matrices and layers, unlike LoRA, which distributes parameters evenly across all modules.


In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset

In [ ]:
from peft import AdaLoraConfig, get_peft_model, TaskType

In [ ]:
import torch

Use on encoder

In [ ]:
model_name = "bert-base-uncased"

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [ ]:
data = {"text": ["I like this movie.", "I dont like this."], "label": [1, 0]}


In [ ]:
from datasets import Dataset

In [ ]:
dataset= Dataset.from_dict(data)

In [ ]:
def preprocess(ex):
    return tokenizer(ex["text"], truncation=True, padding="max_length", max_length=32)

In [ ]:
tokenized_dataset= dataset.map(preprocess)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2
})

parameters from fine tuning, lora metric initilaise of size 8. reduces to 4 finally, init_r is max possible rank during the training. Increase rank to understand the complexity of the data, t_init is the training step,


In [ ]:
adalora_config=AdaLoraConfig(task_type=TaskType.SEQ_CLS,
              r=8,
              target_r=4,
              init_r=12,
              tinit=50,
              tfinal=200, #w ehne to stop
              deltaT=10,
              lora_alpha=16, #higher value, scale
              lora_dropout=0.1 # regularization
              )

 beta1=0.85, beta2=0.85.These are montum. how quickly should rank respond to model learning. Very quickly it will update the rank

In [ ]:
no_of_batches=1000/10

In [ ]:
no_of_batches

100.0

after 50 adjust adalora. adjustes the rank. within the training not per epoch.

delta eVERY 10th
STEP IT WILL UPDATE
lora_alpha is scaling factor

dropput is appling regularization so it doesnt overfit

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model=get_peft_model(model, adalora_config) # pass base model and config. Fine tunned  model

In [ ]:
# device=torch.device=("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model=model.to(device)

In [ ]:
model.print_trainable_parameters()

trainable params: 444,194 || all params: 109,927,996 || trainable%: 0.4041


In [ ]:
training_args=TrainingArguments(output_dir="./results",per_device_eval_batch_size=2, # only 1 batch since we have 2 samples
                                          num_train_epochs=5,
                                          logging_steps=1)

In [ ]:
trainer=Trainer(model=model, args= training_args,train_dataset=tokenized_dataset, eval_dataset= tokenized_dataset)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss
1,1.851400
2,2.002500
3,2.046800
4,1.831000
5,1.918000


TrainOutput(global_step=5, training_loss=1.9299492597579957, metrics={'train_runtime': 9.8539, 'train_samples_per_second': 1.015, 'train_steps_per_second': 0.507, 'total_flos': 165297308160.0, 'train_loss': 1.9299492597579957, 'epoch': 5.0})

In [ ]:
data="This is excellent"
test = tokenizer(data, return_tensors="pt", padding=True, truncation=True)

In [ ]:
output = model(**test)

In [ ]:
pred = torch.argmax(output.logits).item()
if pred==1:
    print("positive")
else:
    print("Negative")

Negative
